<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/09%20-%20Word%20embedding%20e%20Word2Vec/word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Word embedding
Il **Word embedding** è un modello che ci permette di generare una serie di vettori (embedding vectors) ognuno dei quali quantifica una caratteristica delle parole. In questo notebook creeremo una rete neurale artificiale per classsificare recensioni come positive o negative usando il Word Embedding per codificare le recensioni.

In [1]:
!pip install numpy==1.16.2

## Prepariamo i dati
In precedenza abbiamo caricato e preprocessato manualmente l'IMDB Movies Review Dataset, in questo caso utilizziamo direttamente il dataset già preprocessato che ci mette a disposizione Keras.
<br>**ATTENZIONE**<br>
Se caricando il dataset ottieni questo errore:<br>
*ValueError: Object arrays cannot be loaded when allow_pickle=False*
<br>
questo è casuato da un bug nell'ultima versione di keras, per correggerlo esegui il downgrade di numpy usando la cella di codice qui sotto e riavvia il kernel (su Colaboratory seleziona Runtime dalla barra dei comandi e clicca su Restart Runtime).

In [0]:
!pip install numpy==1.16.2

In [22]:
import numpy as np
from keras.datasets import imdb 

num_words = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


Ogni riga della lista con le features corrisponde ad una frase, ogni colonna contiene l'indice di una parola all'interno del vocabolario dell'intero corpus di testo. Il vettore con il target contiene un unico valore che può essere 0 per una recensione negativa o 1 per una recensione positiva.<br> 
Definiamo una funzione che ci permette di ricostruire la frase partendo dagli indici, per farlo abbiamo bisogno del vocabolario che mappa le parole agli indici, possiamo ottenerlo con il metodo *.get_word_index()*.
<br>
**NOTA BENE**
<br>
Gli indici delle parole hanno un'offset di 3, quindi per ottenere l'indice corretto per il vocabolario dovremo sottrarre 3 all'indice della parola contenuto nella frase.

In [58]:
word_index = imdb.get_word_index()
index_word = dict([(value, key) for (key, value) in word_index.items()])

def vec_to_text(x):
  
  text = [index_word.get(i-3,'?') for i in x]
  return " ".join(text)
  
vec_to_text(X_test[0])

"terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite ? so all you madison fans give this a miss"

Ovviamente le recensioni avranno lunghezza differente, calcoliamo la lunghezza della più lunga e della più corta.

In [24]:
longest_review = max(X_train,key=len)
shortest_review = min(X_train,key=len)

print("La review più lunga ha %d parole" % len(longest_review))
print("La review più corta ha %d parole" % len(shortest_review))

La review più lunga ha 2494 parole
La review più corta ha 11 parole


Per rendere le features un buon input per il nostro modello dobbiamo fare in modo che ogni frase abbia la stessa lunghezza, per farlo possiamo usare la funzione *pad_sequences(text)* di keras, che riduce tutte le frasi ad una lunghezza prefissata troncando quelle troppo lunghe e aggiungendo degli zeri a quelle troppo brevi. Usiamo una lunghezza comune di 50 parole.

In [25]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 50

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

X_train.shape

(25000, 50)

## Creiamo il modello
Possiamo utilizzare l'embedding come se fosse uno strato della nostra rete neurale che verrà anch'esso ottimizzato durante la fase di addestramento. Creiamo la rete aggiungendo al primo strato uno strato di embedding, dopodichè aggiungiamo un'altro strato che utilizza la classe Flatten() di keras per convertire la matrice che contiene la rappresentazione vettoriale di una frase in un vettore, unendo tutte le righe una dietro l'altra. Infine aggiungiamo uno strato di output per eseguire la classificazione binaria.

In [37]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Flatten

model = Sequential()

model.add(Embedding(num_words, 50, input_length=maxlen))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 50, 50)            500000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 2500)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 2501      
Total params: 502,501
Trainable params: 502,501
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello e avviamo l'addestramento per 10 epoche.

In [38]:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 61us/step - loss: 0.6890 - acc: 0.5621 - val_loss: 0.6786 - val_acc: 0.6538
Epoch 2/10
20000/20000 [==============================] - 1s 35us/step - loss: 0.6346 - acc: 0.7770 - val_loss: 0.6023 - val_acc: 0.7410
Epoch 3/10
20000/20000 [==============================] - 1s 35us/step - loss: 0.5101 - acc: 0.8193 - val_loss: 0.4969 - val_acc: 0.7790
Epoch 4/10
20000/20000 [==============================] - 1s 37us/step - loss: 0.3932 - acc: 0.8570 - val_loss: 0.4415 - val_acc: 0.7982
Epoch 5/10
20000/20000 [==============================] - 1s 36us/step - loss: 0.3183 - acc: 0.8892 - val_loss: 0.4151 - val_acc: 0.8054
Epoch 6/10
20000/20000 [==============================] - 1s 35us/step - loss: 0.2630 - acc: 0.9158 - val_loss: 0.4053 - val_acc: 0.8076
Epoch 7/10
20000/20000 [==============================] - 1s 36us/step - loss: 0.2191 - acc: 0.9364 - val_loss: 0.4021 - val_acc:

In [39]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 1s 31us/step


[0.40605927324295044, 0.815]

## Otteniamo gli embedding
Se volessimo conoscere gli embedding che il modello genera per una determinata frase possiamo farlo creando un nuovo modello che da in output l'output dell'embedding che abbiamo addestrato. Keras ci da la possibilità di accedere ai singoli strati di un modello utilizzato l'attributo *.layers*

In [41]:
model.layers[0]

Utilizzando le API Funzionali di Keras (ne parleremo più avanti) possiamo creare un modello che prende in input lo stesso input del modello addestrato e da in output l'output dell'embedding.

In [0]:
from keras.models import Model

embedding_model = Model(inputs=model.input, outputs=model.layers[0].output)

Utilizzando il metodo *.predict(x)* otterremo una matrice con la rappresentazione vettoriale di ogni parola della frase.

In [42]:
x = np.array([X_test[0]])

embedding_model.predict(x)

array([[[ 0.30276826,  0.111716  ,  0.17554781, ...,  0.2054588 ,
         -0.23213504,  0.12865287],
        [-0.10857813,  0.00208598, -0.11951214, ..., -0.20470296,
         -0.12110917, -0.09332115],
        [-0.05576379,  0.0109272 , -0.00774001, ...,  0.02854038,
         -0.02343158,  0.02618085],
        ...,
        [ 0.08926877, -0.00070353, -0.05446988, ...,  0.07803367,
         -0.0772426 , -0.10260648],
        [-0.0244258 , -0.0769368 , -0.12284474, ..., -0.02184814,
          0.03313072, -0.04578592],
        [-0.07755034,  0.29535306, -0.04420415, ..., -0.05049261,
         -0.04422899, -0.03766544]]], dtype=float32)